In [1]:
import sys
import os
import pickle

from IPython.display import HTML

# from ipywidgets import interact, interactive, fixed, interact_manual, HBox,HTML

import ipywidgets as widgets
import pandas as pd
import numpy as np

sys.path.append("../")
from scripts.wiki import Wiki,Revision



ModuleNotFoundError: No module named 'scripts.wiki'

# Visualisation for a single article 

All change object  between **revision ids** for an article. Given an **article_name** and a *revision id* shows change object with next *revision id*.

In [ ]:
table_style =     [
    {'selector': 'table', 'props': [('border', "6px double #696969")]},
    {'selector': 'th', 'props': [('border', "2px solid #D3D3D3"), ("font-size", "100%")]},
    {"selector":".data", "props":[("text-align", "justify"), ('border', "1px solid #000"), ('margin', '4px 24px 4px 24px' ), ("font-size", "8pt")]}
] 

deleted_token_style = {"color":"red", "font-weight": "bold","font-size": "100px"}
inserted_token_style = {"color":"blue", "font-weight": "bold","font-size": "100px"}

In [ ]:
### getting the vlaue of the article to visualise data.
article_name = "John_Logie_Baird"

### Show revision contents.

In [ ]:
change_object_dir =  "../data/change objects/"
change_dataframe_path = os.path.join(change_object_dir, article_name+"_change.h5")

change_df=pd.read_hdf(change_dataframe_path, key="data")

In [ ]:
bykau_dir =  "../data/bykau_change_object/"
filename =  f"{article_name}_change.h5"

change_object_file = os.path.join(bykau_dir, filename)
bykau_change_df = pd.read_hdf(change_object_file, key="data")

In [ ]:
# %%time
# change_object_dir =  "../data/change objects/"
# filename = article_name + ".pkl"
# filepath = os.path.join(change_object_dir, filename)
# if os.path.exists(filepath):
#     with open(filepath, "rb") as file:
#         wiki = pickle.load(file)
# else:
#     print("file do not exist")

##### Reading revision list

In [ ]:
content_dir = "../data/content/"
content_file = article_name + ".h5"
content_path = os.path.join(content_dir, content_file)

with pd.HDFStore(content_path, 'r') as store:
    #retrieving all rev list and change object from file
    rev_list = store.get("rev_list")

In [ ]:
def display_article_content(index, rev_list, out):
    with out:
        out.clear_output()
    with out:
        display("Content is being loaded from hard disk please wait.")
    from_rev_id = str(rev_list.loc[index, "id"])
    from_key = "r" + from_rev_id
    to_rev_id = str(rev_list.loc[index+1, "id"])
    to_key = "r" + to_rev_id
    with pd.HDFStore(content_path, 'r') as store:
        from_rev_data = store.get(from_key)
        to_rev_data = store.get(to_key)
    
    #deleted positions
    deleted_positions = change_df.loc[rev_list.loc[index,"id"]].reset_index(drop=True)[["del_start_pos", "del_end_pos"]]
    deleted_positions = deleted_positions[deleted_positions["del_start_pos"]!=-1]
    deleted_positions = deleted_positions.apply(lambda x: np.arange(x["del_start_pos"], x["del_end_pos"]+1) , axis=1).values
    if deleted_positions.size !=0:
        deleted_positions = np.concatenate(deleted_positions, axis=0)
    else: 
        deleted_positions = np.empty(0)
        
    #inserted position
    inserted_positions = change_df.loc[rev_list.loc[index,"id"]].reset_index(drop=True)[["ins_start_pos", "ins_end_pos"]]
    inserted_positions = inserted_positions[inserted_positions["ins_start_pos"]!=-1]
    inserted_positions = inserted_positions.apply(lambda x: np.arange(x["ins_start_pos"], x["ins_end_pos"]+1) , axis=1).values
    if inserted_positions.size !=0:
        inserted_positions = np.concatenate(inserted_positions, axis=0)
    else: 
        inserted_positions = np.empty(0)
        
    #show deleted positions as red in revisions which was changed or from_rev.
    from_html = widgets.HTML(from_rev_data.style.set_caption(" Tok track tokens for revision id "+ from_rev_id).set_table_styles(table_style).
                             set_properties(subset= pd.IndexSlice[deleted_positions,:], **deleted_token_style).render())
    #show inserted positions as green in revisions which was changed or to_rev.
    to_html = widgets.HTML(to_rev_data.style.set_caption(" Tok track tokens to revision id "+ to_rev_id).set_table_styles(table_style).
                           set_properties(subset= pd.IndexSlice[inserted_positions,:], **inserted_token_style ).render())
    
    editor = rev_list.loc[index,"editor"] 
    timestamp = rev_list.loc[index, "timestamp"]
    change_html = widgets.HTML(change_df.loc[rev_list.loc[index,"id"], ["left_token", "ins_tokens", "del_tokens", "right_token"]].reset_index(drop=True).
                               style.set_caption(f"change object created by  {editor} at {timestamp} " ).
                               set_table_styles(table_style).set_properties( subset= pd.IndexSlice[:,"ins_tokens"], **inserted_token_style).
                               set_properties(subset= pd.IndexSlice[:, "del_tokens"], **deleted_token_style ).render())
    if rev_list.loc[index,"id"]  in bykau_change_df.index:
        bykau_change_html = widgets.HTML(bykau_change_df.loc[rev_list.loc[index,"id"], ["left_token", "ins_tokens", "del_tokens", "right_token"]].reset_index(drop=True).
                               style.set_caption(f"bykau et al change object created by  {editor} at {timestamp} " ).
                               set_table_styles(table_style).set_properties( subset= pd.IndexSlice[:,"ins_tokens"], **inserted_token_style).
                               set_properties(subset= pd.IndexSlice[:, "del_tokens"], **deleted_token_style ).render())
    else:
        bykau_change_html = widgets.HTML("<p>revision id do not exist in byaku change object.</p>")
    with out:
        out.clear_output()
    with out:
        display(widgets.HBox([from_html, to_html, change_html, bykau_change_html]))

In [ ]:
drop_down = list(zip(rev_list.id, rev_list.index))

In [ ]:
outp = widgets.Output(layout={'border': '1px solid black'})

## This shows change object between two revision ids
**Please rerun next two cells each time page is reloaded**



In [ ]:
widgets.interact(display_article_content, index=drop_down, rev_list= widgets.fixed(rev_list), out=widgets.fixed(outp));

In [ ]:
outp